In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

import matplotlib.pyplot as plt
from scipy import stats

In [3]:
df = pd.read_csv('055550.KS.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,2000-01-04,18688.708984,20234.212891,18308.277344,19259.357422,197145699
1,2000-01-05,19021.585938,19592.234375,18070.507812,18664.931641,186396333
2,2000-01-06,18807.593750,19497.126953,16406.119141,17119.427734,184079807
3,2000-01-07,18546.046875,18878.923828,17618.744141,18070.507812,256406581
4,2000-01-10,18783.816406,19211.802734,18546.046875,18926.478516,988218404


In [4]:
df.index = pd.DatetimeIndex(df["Date"])
df.drop(["Date"],axis=1,inplace=True)
df

,Open,High,Low,Close,Volume
Date,,,,,
2000-01-04,18688.708984,20234.212891,18308.277344,19259.357422,197145699
2000-01-05,19021.585938,19592.234375,18070.507812,18664.931641,186396333
2000-01-06,18807.593750,19497.126953,16406.119141,17119.427734,184079807
2000-01-07,18546.046875,18878.923828,17618.744141,18070.507812,256406581
2000-01-10,18783.816406,19211.802734,18546.046875,18926.478516,988218404
...,...,...,...,...,...
2023-11-24,37050.000000,37100.000000,36750.000000,36750.000000,404462
2023-11-27,36500.000000,37000.000000,36450.000000,36750.000000,601759
2023-11-28,37050.000000,37100.000000,36600.000000,37000.000000,742029


In [8]:
n_train = int(0.8*len(df))
train = df[:n_train]
test = df[n_train:]

In [9]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 21.0.1+12-LTS-29, mixed mode, sharing)
  Starting server from C:\Users\cho03\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\cho03\AppData\Local\Temp\tmp13nbrbap
  JVM stdout: C:\Users\cho03\AppData\Local\Temp\tmp13nbrbap\h2o_cho03_started_from_python.out
  JVM stderr: C:\Users\cho03\AppData\Local\Temp\tmp13nbrbap\h2o_cho03_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_cho03_9gogd6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.944 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [10]:
df_train = h2o.H2OFrame(train)
df_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
df_train.describe()

Rows:4798
Cols:5

,Open,High,Low,Close,Volume
type,real,real,real,real,int
mins,6039.35400390625,6562.447265625,6039.35400390625,6182.015625,0.0
mean,35111.13958188135,35593.57517526427,34597.64017193045,35105.77520377945,226625571.08378398
maxs,63722.31640625,64197.85546875,61820.15625,62961.453125,12116862397.0
sigma,13946.780331858796,14033.220582265014,13816.370397535182,13943.646237234801,822901955.1070399
zeros,0,0,0,0,111
missing,0,0,0,0,0
0,18688.708984375,20234.212890625,18308.27734375,19259.357421875,197145699.0
1,19021.5859375,19592.234375,18070.5078125,18664.931640625,186396333.0
2,18807.59375,19497.126953125,16406.119140625,17119.427734375,184079807.0


In [13]:
y = 'Close'
X = df_train.columns
X.remove(y)

In [14]:
X

['Open', 'High', 'Low', 'Volume']

In [16]:
aml = H2OAutoML(max_runtime_secs = 600, seed = 42)
aml.train(x = X, y = y, training_frame = df_train, leaderboard_frame = df_test)

AutoML progress: |
01:10:58.478: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/9
# GBM base models (used / total),3/5
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [21]:
lb = aml.leaderboard
lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_2_AutoML_1_20231203_11058,274.499,75349.9,207.342,0.00781162,75349.9
StackedEnsemble_BestOfFamily_3_AutoML_1_20231203_11058,274.512,75356.9,207.345,0.00781182,75356.9
DeepLearning_grid_1_AutoML_1_20231203_11058_model_2,279.976,78386.7,212.715,0.00798181,78386.7
GBM_grid_1_AutoML_1_20231203_11058_model_112,287.07,82409.4,219.184,0.00823968,82409.4
GBM_grid_1_AutoML_1_20231203_11058_model_73,287.216,82493.3,216.961,0.00825064,82493.3
GBM_grid_1_AutoML_1_20231203_11058_model_35,288.916,83472.5,220.587,0.00826978,83472.5
DeepLearning_1_AutoML_1_20231203_11058,289.241,83660.5,217.278,0.00821751,83660.5
GBM_grid_1_AutoML_1_20231203_11058_model_12,289.323,83707.8,218.468,0.00833077,83707.8
GBM_grid_1_AutoML_1_20231203_11058_model_102,290.667,84487.2,219.042,0.00830618,84487.2
GBM_grid_1_AutoML_1_20231203_11058_model_2,291.496,84969.9,220.787,0.00834935,84969.9


In [18]:
leader_model = aml.leader

In [19]:
df_test_predict = leader_model.predict(df_test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [20]:
df_test_predict.head(30)

predict
40860.8
40605.5
40676.4
40553.8
41682.7
41930.3
42249.8
42160.1
42083.7
43252.5


In [25]:
df_results = pd.DataFrame()
df_results['ground_truth'] = test['Close'].reset_index(drop=True)
df_results['predictions'] = h2o.as_list(df_test_predict,use_pandas=True)
df_results

,ground_truth,predictions
0,40750.0,40860.842491
1,40600.0,40605.487575
2,40600.0,40676.397517
3,40600.0,40553.807960
4,41950.0,41682.739079
...,...,...
1195,36750.0,36897.908578
1196,36750.0,36795.954471
1197,37000.0,36826.925820
1198,36300.0,36638.248388
